In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pyemu
from pathlib import Path
import os, shutil

In [ ]:
ies_dir = Path('../tmprun/')

## take a look at the prior MC 

In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
phi0 = phi[0]

In [ ]:
phi0.hist(bins=50)


## let's check out the observations

In [ ]:
pst = pyemu.Pst(str(ies_dir / 'mv.ies.pst'))

In [ ]:
obsdat = pst.observation_data.copy()
obsdat.sample(5)

In [ ]:
citer = 3
obs = pd.read_csv(ies_dir / 'mv.ies.0.obs.csv', index_col=0)
obs_final = pd.read_csv(ies_dir / f'mv.ies.{citer}.obs.csv', index_col=0)

In [ ]:
with PdfPages('allobs.pdf') as outpdf:
    for cc in obs.columns:
        plt.figure()
        ax = obs[cc].hist(bins=20, color='grey', alpha=.5, density=1)
        obs_final[cc].hist(bins=20, color='blue', density=1)
        ax.axvline(obsdat.loc[cc].obsval, color='orange')
        plt.title(cc)
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
for cc in phi.columns:
    phi.loc[phi[cc]>1e6,cc] = np.nan

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,5))
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[0])
phi.loc['base'].plot(color='b', ax=ax[0])
ax[0].set_yscale('log')
ax[0].set_title('log-scale $\\Phi$')
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[1])
phi.loc['base'].plot(color='b', ax=ax[1])
ax[1].set_title('linear-scale $\\Phi$');


# Visualize a single set of fields

In [ ]:
tmpdir = Path('../scratch')
if tmpdir.exists():
    shutil.rmtree(tmpdir)
shutil.copytree(ies_dir, tmpdir)

In [ ]:
pst = pyemu.Pst(str(tmpdir / 'mv.ies.pst'))

In [ ]:
pars = pd.read_csv(ies_dir / 'mv.ies.3.par.csv', index_col=0)
pars

In [ ]:
pst.control_data.noptmax=0

### good to check out the base realization, but could choose another

In [ ]:
real_to_plot = 'base'

In [ ]:
pst.parameter_data.loc[pars.loc[real_to_plot].index, 'parval1'] = pars.loc[real_to_plot]

In [ ]:
pst.write(str(tmpdir / 'testrun.pst'), version=2)

In [ ]:
pyemu.os_utils.run('pestpp-ies testrun.pst', str(tmpdir))

## viz the k fields

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5))
a = ax[0].imshow(np.loadtxt(tmpdir / 'k_aq.ref'))
plt.colorbar(a)
ax[0].set_title('Aquifer KH')
c = ax[1].imshow(np.loadtxt(tmpdir / 'k_clay.ref'))
ax[1].set_title('Clay Layer KH')
plt.colorbar(c)
plt.suptitle(f'realization: {real_to_plot}, iteration: {citer}')

In [ ]:
plt.imshow(np.loadtxt('../pest_background_files/k_aq.ref'))
plt.colorbar()

In [ ]:
pars.kaniso.hist(bins=50)
plt.title('Anisotropy: aquifer');

In [ ]:
pars.kaniso_clay.hist(bins=50)
plt.title('Anisotropy: clay layer');

In [ ]:
pars.rch.hist(bins=50)
plt.title('Recharge');